In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import remove_too_long,type_score
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache


In [38]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [39]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [40]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0]})
df_ipcc_group=df_ipcc_group.reset_index()

In [41]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [23]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [24]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: {'chapter': x.replace('chap_','') for x in row['chap']}, axis=1)

In [25]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'ipbes': lambda x: list(x), 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

In [26]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [27]:
df_ipbes_group

,doi,ipbes,countries,freq
0,10.1001/archpedi.161.7.718,[{'chapter': '5'}],None,1
1,10.1002/(sici)1099-0771(199612)9:4<231::aid-bd...,[{'chapter': '4'}],[US],1
2,10.1002/16-0188.1,"[{'chapter': '1'}, {'chapter': '5'}]",[US],2
3,10.1002/2013ef000188,[{'chapter': '1'}],[FR],1
4,10.1002/2013gb004654,[{'chapter': '2'}],[DE],1
...,...,...,...,...
6600,10.7930/j0x63jt0,[{'chapter': 'y'}],None,1
6601,10.9755/ejfa.v25i1.15376,[{'chapter': '3'}],[US],1
6602,10.9758/cpn.2015.13.3.239,[{'chapter': '3'}],[TR],1
6603,10.9774/gleaf.978-1-907643-06-4_5,[{'chapter': '1'}],None,1


In [42]:
df_ipcc_group

,doi,ipcc,countries
0,10.1001/jama.2012.108940,"[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6,...",[US]
1,10.1001/jama.2014.13186,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...","[GB, US]"
2,10.1001/jama.2014.15715,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US]
3,10.1001/jama.2020.1097,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US]
4,10.1001/jama.2020.15398,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US]
...,...,...,...
53107,10.9753/icce.v36.risk.59,"[{'name': 'wg2_chap_16', 'wg': '2', 'chap': 16...","[GB, JP, US]"
53108,10.9770/jesi.2017.4.4(13),"[{'name': 'wg2_chap_18', 'wg': '2', 'chap': 18...",None
53109,10.9774/gleaf.3062.2001.wi.00007,"[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4,...",None
53110,10.9774/gleaf.4700.2015.ju.00009,"[{'name': 'wg2_chap_18', 'wg': '2', 'chap': 18...",None


In [43]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

,doi,ipcc,countries_x,ipbes,countries_y,freq
0,10.1001/jama.2012.108940,"[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6,...",[US],NaN,NaN,NaN
1,10.1001/jama.2014.13186,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...","[GB, US]",NaN,NaN,NaN
2,10.1001/jama.2014.15715,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US],NaN,NaN,NaN
3,10.1001/jama.2020.1097,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US],NaN,NaN,NaN
4,10.1001/jama.2020.15398,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US],NaN,NaN,NaN
...,...,...,...,...,...,...
58540,10.7930/j0x63jt0,NaN,NaN,[{'chapter': 'y'}],None,1.0
58541,10.9755/ejfa.v25i1.15376,NaN,NaN,[{'chapter': '3'}],[US],1.0
58542,10.9758/cpn.2015.13.3.239,NaN,NaN,[{'chapter': '3'}],[TR],1.0
58543,10.9774/gleaf.978-1-907643-06-4_5,NaN,NaN,[{'chapter': '1'}],None,1.0


In [44]:
update_bso_publications()

2024-02-29 16:20:01,826 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files
c:\Users\haallat\Documents\teds\code_utils\BSO.py:34: DtypeWarning: Columns (17,18,21,22,23,24,25,27,28,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bso = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz', sep=';')


2024-02-29 16:22:07,343 | code_utils.BSO | DEBUG | writing 1812497 dois info from bso publications


DEBUG:code_utils.BSO:writing 1812497 dois info from bso publications


In [45]:
bso=pd.DataFrame(get_bso_publications())

In [46]:
df_bso=bso.transpose().reset_index()

In [47]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [48]:
df_bso

,doi,rors,bso_local_affiliations
0,10.3917/capo.005.0103,[057qpr032],[130018351]
1,10.4000/champpenal.10450,[057qpr032],[130018351]
2,10.7193/dm.100.79.96,[057qpr032],[130018351]
3,10.1080/09500340.2015.1079655,[057qpr032],[130018351]
4,10.1056/nejmc1604852,[057qpr032],[130018351]
...,...,...,...
1812492,10.1101/2020.04.15.044065,[],[]
1812493,10.1101/147884,[],[]
1812494,10.1162/qss_a_00146,[],[]
1812495,10.1364/optica.397172,[02en5vm52],"[180089013, 130023385]"


In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [ ]:
def get_countries(row):
    if (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)==False):
        return None
    elif (isinstance(row.countries_x,list))&(isinstance(row.countries_y,list)==False):
        return row.countries_x
    elif (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)):
        return row.countries_y
    else:
        return row.countries_x

In [ ]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [ ]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [ ]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [ ]:
df_iso=df_iso.rename(columns={'code':'countries'})

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_iso, on='countries', how='left')
del df_ipcc_ipbes_bso['countries']
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso.rename(columns={'name':'countries'})

In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']

In [ ]:
len(df_elastic_input)

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'